In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50

# Specify the folders


In [2]:

path = r"D:\Github\AI\Computer Vision\Face Recogniton System\Store_images_model_required_to_run"

# List images in the folders by explicit names


In [3]:
def create_siamese_model(input_shape):
    input_layer = layers.Input(shape=input_shape)

    # Load a pre-trained ResNet50 model
    base_model = ResNet50(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

    # Feature extraction with ResNet50
    x = base_model(input_layer)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)

    return Model(input_layer, x)

# Set input shape and define the model
input_shape = (224, 224, 3)
anchor_input = layers.Input(name="anchor", shape=input_shape)
positive_input = layers.Input(name="positive", shape=input_shape)

siamese_network = create_siamese_model(input_shape)

encoded_anchor = siamese_network(anchor_input)
encoded_positive = siamese_network(positive_input)

In [4]:
L1_distance = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([encoded_anchor, encoded_positive])
output = layers.Dense(1, activation="sigmoid")(L1_distance)

model = Model(inputs=[anchor_input, positive_input], outputs=output)

In [5]:
weights_path = r"D:\Github\AI\Computer Vision\Face Recogniton System\siamese_model_weights.weights.h5"

In [6]:
# weights_path = r"D:\Github\AI\Computer Vision\Face Recogniton System\Project_Code\siamese_model_weights.weights.h5"

# Load weights into the model
model.load_weights(weights_path)

In [7]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img,(224,224))
    img = img/255.0
    return img

In [8]:
def Verify(model , detection_threshold , verfication_threshold):
    results = []

    for image in os.listdir(os.path.join(path, "Verification_images")):
        input_img = preprocess(os.path.join(path, "Input_images","input_image.jpg"))
        validation_img = preprocess(os.path.join (path,"Verification_images",image))

        result = model.predict(list(np.expand_dims([input_img,validation_img],axis=1)))
        results.append(result)

        


    detection = np.sum(np.array(results)>detection_threshold)
    
    verification = detection / len (os.listdir(os.path.join (path ,"Verification_images")))

    print("Detetction " , detection)
    print(len(os.listdir(os.path.join (path ,"Verification_images"))))

    verified = verification >verfication_threshold
    print(results)

    return result, verified

In [11]:
cap = cv2.VideoCapture("https://10.26.83.138:8080/video")
# cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.resize(frame , (400,400))
    # frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
        cv2.imwrite(os.path.join(path, 'Input_images', 'input_image.jpg'), frame)
        print("The Images is write")
        # Run verification
        results, verified = Verify(model, 0.5, 0.4)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

The Images is write
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━